In [ ]:
import os
import sys
import boto3
import datetime
import pandas as pd

from pathlib import Path
from dotenv import load_dotenv

from shared import *

load_dotenv()


In [ ]:
def merge(workspace='./data/', output='merged/', download=True, target='archive', merge='fraud'):
    if download:
        # get all the data from S3
        download_bucket_folder(target)
        download_bucket_folder(merge)

    # load the target data
    input_dir = workspace + target
    files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
    
    target_df = load_transactions(files)

    target_df = target_df.sort_values('TRANSACTION_ID')
    target_df.reset_index(drop=True, inplace=True)
    target_df = target_df.drop_duplicates()

    # load the "to be merge" data
    input_dir = workspace + merge
    files = [os.path.join(input_dir, f) for f in os.listdir(input_dir)]
    
    merge_df = load_transactions(files)

    merge_df = merge_df.sort_values('TRANSACTION_ID')
    merge_df.reset_index(drop=True, inplace=True)
    merge_df = merge_df.drop_duplicates()

    # merge the prediction data
    for index, row in merge_df.iterrows():
        idx = target_df.loc[target_df['TRANSACTION_ID'] == row['TRANSACTION_ID']]
        if len(idx) > 0:
            target_df.loc[idx.index, ['TX_FRAUD_SCENARIO']] = row['TX_FRAUD_SCENARIO']
            target_df.loc[idx.index, ['TX_FRAUD_PREDICTION']] = 1
            target_df.loc[idx.index, ['TX_FRAUD_PROBABILITY']] = 1.0

    # sort the result chronological
    target_df = target_df.sort_values('TX_DATETIME')
    target_df.reset_index(drop=True, inplace=True)

    # add columns that makes sorting and grouping easier
    start_date = target_df['TX_DATETIME'].min().strftime("%Y-%m-%d")
    sd = datetime.datetime.strptime(start_date, "%Y-%m-%d")

    target_df['TX_TIME_SECONDS'] = pd.to_numeric(target_df['TX_DATETIME']) / 1000000000
    target_df['TX_TIME_SECONDS'] = target_df['TX_TIME_SECONDS'].astype(int)

    target_df['TX_TIME_DAYS'] = tx_df['TX_DATETIME'] - sd
    target_df['TX_TIME_DAYS'] = tx_df['TX_TIME_DAYS'].apply(lambda x: x.days)

    # save the merged data, grouped by day
    save_transactions_day(target_df, start_date, output_dir=workspace + output)


In [ ]:
merge()